In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC # ETL Process for Customer 360 View
# MAGIC This notebook performs an ETL process to create a comprehensive Customer 360 view by integrating data from multiple sources.

# COMMAND ----------

import logging
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, DoubleType, IntegerType

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# COMMAND ----------

try:
    # Load data from Unity Catalog tables
    logger.info("Loading data from Unity Catalog tables.")
    policy_df = spark.table("catalog.source_db.policy")
    claims_df = spark.table("catalog.source_db.claims")
    demographics_df = spark.table("catalog.source_db.demographics")
    scores_df = spark.table("catalog.source_db.scores")
    aiml_insights_df = spark.table("catalog.source_db.aiml_insights")

# COMMAND ----------

    # Data Selection and Filtering
    logger.info("Selecting relevant fields from each dataset.")
    demographics_selected = demographics_df.select(
        "Customer_ID", "Customer_Name", "Email", "Phone_Number", "Address", "City", "State", "Postal_Code",
        F.col("Date_of_Birth").cast(DateType()), "Gender", "Marital_Status", "Occupation", "Income_Level", "Customer_Segment"
    )
    claims_selected = claims_df.select(
        "Claim_ID", "Policy_ID", F.col("Claim_Date").cast(DateType()), "Claim_Type", "Claim_Status",
        F.col("Claim_Amount").cast(DoubleType()), F.col("Claim_Payout").cast(DoubleType())
    )
    policy_selected = policy_df.select(
        "policy_id", "customer_id", "policy_type", "policy_status",
        F.col("policy_start_date").cast(DateType()), F.col("policy_end_date").cast(DateType()),
        F.col("policy_term").cast(IntegerType()), F.col("policy_premium").cast(DoubleType()),
        F.col("total_premium_paid").cast(DoubleType()), "renewal_status", "policy_addons"
    )

# COMMAND ----------

    # Data Integration
    logger.info("Joining datasets based on common identifiers.")
    joined_df = demographics_selected.join(policy_selected, demographics_selected.Customer_ID == policy_selected.customer_id, "inner") \
                                     .join(claims_selected, policy_selected.policy_id == claims_selected.Policy_ID, "inner")

# COMMAND ----------

    # Data Aggregation
    logger.info("Computing aggregate metrics.")
    aggregated_df = joined_df.groupBy("Customer_ID").agg(
        F.count("Claim_ID").alias("Total_Claims"),
        F.countDistinct("policy_id").alias("Policy_Count"),
        F.max("Claim_Date").alias("Recent_Claim_Date"),
        F.avg("Claim_Amount").alias("Average_Claim_Amount")
    )

# COMMAND ----------

    # Custom Calculations
    logger.info("Implementing custom calculations.")
    final_df = aggregated_df.withColumn("Age", F.datediff(F.current_date(), F.to_date("Date_of_Birth")) / 365) \
                            .withColumn("Claim_To_Premium_Ratio", F.when(F.col("total_premium_paid") != 0, F.col("Claim_Amount") / F.col("total_premium_paid")).otherwise(0)) \
                            .withColumn("Claims_Per_Policy", F.when(F.col("Policy_Count") != 0, F.col("Total_Claims") / F.col("Policy_Count")).otherwise(0)) \
                            .withColumn("Retention_Rate", F.lit(0.85)) \
                            .withColumn("Cross_Sell_Opportunities", F.lit("Multi-Policy Discount, Home Coverage Add-on")) \
                            .withColumn("Upsell_Potential", F.lit("Premium Vehicle Coverage"))

# COMMAND ----------

    # Comprehensive Data Consolidation
    logger.info("Combining all data sources into a single view.")
    customer_360_df = final_df.join(scores_df, "Customer_ID", "inner") \
                              .join(aiml_insights_df, "Customer_ID", "inner")

# COMMAND ----------

    # Output Generation
    logger.info("Writing the final output to Unity Catalog table.")
    customer_360_df.write.format("delta").mode("overwrite").saveAsTable("catalog.target_db.customer_360")

    logger.info("ETL process completed successfully.")

except Exception as e:
    logger.error("An error occurred during the ETL process.", exc_info=True)
